Для любых видео восстановить траекторию движения (t вектор). Выполнить визуализацию. 
Определить параметры которые влияют на "точность" определения вектора t
Использовать решение на базе нейронных сетей. 
Любые идеи. 

***slam прикрутить. 
Видео 1 но что бы работало.    

Анализ задания и пути решения.
Грузим из интернета качественное видео полета дрона. Сохраняем кусок не менее 2 мин.
В расчете используем предобученную нейросеть из модуля kornia, которая ищет соответствие ключевых точек двух наборов, а также извлекает дескрипторы для найденного набора ключевых точек.
Сами ключевые точки можно искать с помощью детектора sift или orb.
Выводим траекторию на трехмерный график.


In [1]:
# Импорт необходимых библиотек
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import torch
import kornia
import kornia.feature as KF
from kornia_moons.feature import laf_from_opencv_SIFT_kpts, cv2_matches_from_kornia

# Фильтрация совпавших особенностей из двух изображений по расстоянию между лучшими совпадениями
def filter_matches_distance(matches, dist_threshold):
    filtered_match = []
    for m, n in matches:
        if m.distance <= dist_threshold * n.distance:
            filtered_match.append(m)
    return filtered_match

# Сопоставление особенностей из двух изображений
def match_features(des1, des2, matching='BF', detector='sift', sort=True, k=2):
    if matching == 'BF':
        if detector == 'sift':
            matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        elif detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING, crossCheck=False)
        matches = matcher.knnMatch(des1, des2, k=k)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        matches = matcher.knnMatch(des1, des2, k=k)
    if sort:
        matches = sorted(matches, key=lambda x: x[0].distance)
    return matches

# Загрузка нейронной сети HardNet
model = KF.HardNet(True)

# Функция для локального дескриптора
def local_descriptor_function(image, key_points, model):
    with torch.no_grad():
        model.eval()
        timg = kornia.image_to_tensor(image, False).float() / 255.
        lafs = laf_from_opencv_SIFT_kpts(key_points)
        affine = KF.LAFAffNetShapeEstimator(True)
        orienter = KF.LAFOrienter(32, angle_detector=KF.OriNet(True))
        orienter.eval()
        affine.eval()
        lafs_new = orienter(affine(lafs, timg), timg)
        patches = KF.extract_patches_from_pyramid(timg, lafs_new, 32)
        B, N, CH, H, W = patches.size()
        descs = model(patches.view(B * N, CH, H, W)).view(B * N, -1)
    return descs.detach().cpu().numpy()

# Функция для сопоставления точек
def points_matching_function(img1, img2, model):
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    detector_type = cv2.ORB_create()
    key_points_1 = detector_type.detect(img1, None)
    key_points_2 = detector_type.detect(img2, None)
    descriptor_1 = local_descriptor_function(img1, key_points_1, model)
    descriptor_2 = local_descriptor_function(img2, key_points_2, model)
    dists, idxs = KF.match_smnn(torch.from_numpy(descriptor_1), torch.from_numpy(descriptor_2), 0.75)
    tentatives = cv2_matches_from_kornia(dists, idxs)
    return key_points_1, key_points_2, tentatives

# Основной код для обработки видео и вычисления траектории
video_path = 'drone.mp4'  # имя файла (и путь, если необходимо)
myvid = cv2.VideoCapture(video_path)  # загрузка в cv2
if not myvid.isOpened():
    print("Видео файл не найден")
else:
    print("Видео открыто")

# Матрица камеры K
CameraMatrix = np.array([[3000, 0., 960],
                         [0., 3000, 540],
                         [0., 0., 1.]], dtype=np.float32)

sift = cv2.SIFT_create()
poses = []  # список для поз камеры
positions = [np.array([0, 0, 0])]  # список для позиций камеры

# Чтение первого кадра видео
ret, frame = myvid.read()
img1 = frame
i = 0
interval = 20  # интервал между рассматриваемыми кадрами

while True:  # i < 1000:
    i += 1
    ret, frame = myvid.read()  # чтение следующего кадра
    if ret and i % interval == 0:  # каждый 5-й кадр
        print(f'Current frame is {i}')
        img2 = frame
        try:
            key_points_1, key_points_2, matches = points_matching_function(img1, img2, model)
            pts1 = np.float32([key_points_1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            pts2 = np.float32([key_points_2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
            F, mask = cv2.findEssentialMat(pts1, pts2, CameraMatrix, method=cv2.LMEDS, prob=0.999, threshold=1.0)
            a, R, t, b = cv2.recoverPose(F, pts1, pts2, CameraMatrix, mask)
            poses.append((R, t))
            current_position = positions[-1]
            new_position = current_position + np.dot(R, t).T[0]
            positions.append(new_position)
            img1 = img2
        except Exception as e:
            print(f"Ошибка: {e}")
            pass
    elif ret and i % interval != 0:
        pass
    else:
        print('Видео - всё')
        myvid.release()  # освобождение ресурса видео после окончания просмотра
        break  # выход из цикла

# Создание траектории и ориентации камеры
def create_trajectory(poses):
    trajectory = [np.array([0, 0, 0])]
    current_pose = np.eye(4)
    pose_cam = [np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])]

    for R, t in poses:
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = t.T
        current_pose = np.dot(current_pose, T)
        trajectory.append(current_pose[:3, 3])
        pose_cam.append(current_pose[:3, :3])

    return np.array(trajectory), np.array(pose_cam)

trajectory, pose_cam = create_trajectory(poses)

# Создание фигуры для графика
fig = go.Figure()

# Добавление траектории камеры
fig.add_trace(go.Scatter3d(
    x=trajectory[1:, 0],
    y=trajectory[1:, 1],
    z=trajectory[1:, 2],
    mode='lines+markers',
    marker=dict(size=3, color='blue'),
    line=dict(color='blue', width=2),
    name='Camera Trajectory'
))

# Добавление ориентации камеры в каждой точке траектории
for i, (R, t) in enumerate(zip(pose_cam, trajectory)):
    camera_direction = R @ np.array([0, 0, 1])  # Направление оси Z камеры
    camera_direction_end = t + camera_direction * 0.5  # Конец вектора направления

    fig.add_trace(go.Scatter3d(
        x=[t[0], camera_direction_end[0]],
        y=[t[1], camera_direction_end[1]],
        z=[t[2], camera_direction_end[2]],
        mode='lines',
        line=dict(color='green', width=2),
        name=f'Camera Direction {i}' if i == 0 else None,
        showlegend=False if i > 0 else True
    ))

# Настройка макета графика
fig.update_layout(
    title='Camera Motion Trajectory',
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    showlegend=True
)

# Показать график
fig.show()


Видео открыто
Current frame is 20


C:\Users\1neon\AppData\Local\Temp\ipykernel_11276\4241881468.py:47: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.
  affine = KF.LAFAffNetShapeEstimator(True)


Current frame is 40
Current frame is 60
Current frame is 80
Current frame is 100
Current frame is 120
Current frame is 140
Current frame is 160
Current frame is 180
Current frame is 200
Current frame is 220
Current frame is 240
Current frame is 260
Current frame is 280
Current frame is 300
Current frame is 320
Current frame is 340
Current frame is 360
Current frame is 380
Current frame is 400
Current frame is 420
Current frame is 440
Current frame is 460
Current frame is 480
Current frame is 500
Current frame is 520
Current frame is 540
Current frame is 560
Current frame is 580
Current frame is 600
Current frame is 620
Current frame is 640
Current frame is 660
Current frame is 680
Current frame is 700
Current frame is 720
Current frame is 740
Current frame is 760
Current frame is 780
Current frame is 800
Current frame is 820
Current frame is 840
Current frame is 860
Current frame is 880
Current frame is 900
Current frame is 920
Current frame is 940
Current frame is 960
Current frame is

In [60]:
quit # сброс всех переменных для повторного запуска программы

# загрузка необходимых библиотек и двух изображений
import cv2
import numpy as np
import matplotlib.pyplot as plt

def filter_matches_distance(matches, dist_threshold):
    """
    Фильтрация совпавших особенностей из двух изображений по расстоянию между лучшими совпадениями

    Аргументы:
    match -- список совпавших особенностей из двух изображений
    dist_threshold -- максимальное допустимое относительное расстояние между лучшими совпадениями, (0.0, 1.0)

    Возвращает:
    filtered_match -- список хороших совпадений, удовлетворяющих порогу расстояния
    """
    filtered_match = []
    for m, n in matches:
        if m.distance <= dist_threshold * n.distance:
            filtered_match.append(m)

    return filtered_match


Используем kornia по модифицированному нами методу В.Шанчука


Реализуем функцию "local_descriptor_function()", которая принимает на вход следующие аргументы:
1) "image" - изображение grayscale
2) "key_points" - массив ключевых точек изображения
3) "model" - модель определения дескриптора.


In [61]:
!pip install kornia
!pip install kornia-moons


In [62]:

import kornia
import kornia.feature as KF
from kornia_moons.feature import laf_from_opencv_SIFT_kpts, cv2_matches_from_kornia
import torch

model = KF.HardNet(True) # loading neural net model

def local_descriptor_function(image, key_points, model):
  with torch.no_grad():
    model.eval()
    # timg = kornia.color.rgb_to_grayscale(kornia.image_to_tensor(image, False).float()) / 255.
    timg = kornia.image_to_tensor(image, False).float() / 255.
    lafs = laf_from_opencv_SIFT_kpts(key_points)
    affine = KF.LAFAffNetShapeEstimator(True)
    orienter = KF.LAFOrienter(32, angle_detector = KF.OriNet(True))
    orienter.eval()
    affine.eval()
    lafs_new = orienter(affine(lafs, timg), timg)
    patches = KF.extract_patches_from_pyramid(timg, lafs_new, 32)
    B, N, CH, H, W = patches.size()
    descs = model(patches.view(B * N, CH, H, W)).view(B * N, -1)
  return descs.detach().cpu().numpy()

Затем реализуем функцию "points_matching_function()", которая принимает на вход следующие аргументы:
1) "image1" - трехмерный массив cv2, представляющий первое изображение BGR
2) "image2" - трехмерный массив cv2, представляющий второе изображение BGR
3) "model" - модель определения дескриптора.
Данная функция первоначально выполняла расчет матрицы аффинного преобразования для набора ключевых точек и дексрипторов двух изображений.
Мы переделали ее для расчета только ключевых точек и соответствий между ними

In [63]:
def points_matching_function(img1, img2, model):
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
  detector_type = cv2.ORB_create()
  key_points_1 = detector_type.detect(img1, None)
  key_points_2 = detector_type.detect(img2, None)
  descriptor_1 = local_descriptor_function(img1, key_points_1, model)
  descriptor_2 = local_descriptor_function(img2, key_points_2, model)
  dists, idxs = KF.match_smnn(torch.from_numpy(descriptor_1), torch.from_numpy(descriptor_2), 0.75)
  tentatives = cv2_matches_from_kornia(dists, idxs)
  return key_points_1, key_points_2, tentatives

Загружаем видео и считываем попарно кадр и следующий кадр. Рассчитываем позицию камеры, если номер следующего считанного кадра кратен десяти.

In [64]:
video_path = 'drone.mp4'  # имя файла (и путь, если необходимо)
myvid = cv2.VideoCapture(video_path) # загрузка в cv2
# Проверка, успешно ли открыто видео
if not myvid.isOpened():
     print("Видео файл не найден")
else:
     print("Видео открыто") 

# матрица камеры K
CameraMatrix = np.array([[3000 ,   0.    , 960],
       [  0.    , 3000 , 540 ],
      [  0.    ,   0.    ,   1.]], dtype=np.float32)

sift = cv2.SIFT_create()
poses = [] # список для поз камеры
# Список для позиций камеры
positions = [np.array([0, 0, 0])]

# Чтение первого кадра видео
ret, frame = myvid.read()
img1 = frame # cv2.cvtColor(frame, cv2. COLOR_BGR2GRAY)
i = 0
interval = 20 #интервал между рассматриваемыми кадрами, например, интервал = 5 для учета каждого 5-го кадра
while True: # i < 1000:
     i += 1
     ret, frame = myvid.read() # чтение следующего кадра          
     if ret and i % interval == 0: # каждый 5-й кадр
         print(f'Current frame is {i}')
         img2 = frame # cv2.cvtColor(frame, cv2. COLOR_BGR2GRAY)
         try:                  
             # keypoints1, keypoints2, matches = match_features_sp(img1, img2)
             keypoints1, keypoints2, matches = points_matching_function(img1, img2, model)             
             # matches = filter_matches_distance(matches, 0.7)             
             pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
             pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
             F, mask = cv2.findEssentialMat(pts1, pts2, CameraMatrix, method=cv2.LMEDS, prob = 0.999, threshold = 1.0)     
             a, R, t, b = cv2.recoverPose(F, pts1, pts2, CameraMatrix, mask)
             # print(R)
             poses.append((R, t))

             # Обновить позицию камеры
             current_position = positions[-1]
             new_position = current_position + np.dot(R, t).T[0]
             positions.append(new_position)
             # Сохранение текущего кадра как предыдущего
             img1 = img2 
         except:
              pass
     elif ret and i % interval != 0:
          pass
     else:
          print('The video is over')
          myvid.release()        # освобождение ресурса видео после окончания просмотра
          # cv2.destroyAllWindows()  # закрытие всех окон
          break                    # выход из цикла



Видео открыто
Current frame is 20


C:\Users\1neon\AppData\Local\Temp\ipykernel_16584\140551588.py:14: DeprecationWarning:

`LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.



Current frame is 40
Current frame is 60
Current frame is 80
Current frame is 100
Current frame is 120
Current frame is 140
Current frame is 160
Current frame is 180
Current frame is 200
Current frame is 220
Current frame is 240
Current frame is 260
Current frame is 280
Current frame is 300
Current frame is 320
Current frame is 340
Current frame is 360
Current frame is 380
Current frame is 400
Current frame is 420
Current frame is 440
Current frame is 460
Current frame is 480
Current frame is 500
Current frame is 520
Current frame is 540
Current frame is 560
Current frame is 580
Current frame is 600
Current frame is 620
Current frame is 640
Current frame is 660
Current frame is 680
Current frame is 700
Current frame is 720
Current frame is 740
Current frame is 760
Current frame is 780
Current frame is 800
Current frame is 820
Current frame is 840
Current frame is 860
Current frame is 880
Current frame is 900
Current frame is 920
Current frame is 940
Current frame is 960
Current frame is

Визуализируем результат


In [65]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def create_trajectory(poses):
    trajectory = [np.array([0, 0, 0])]
    current_pose = np.eye(4)
    pose_cam = [np.array([[0, 0, 0],[0, 0, 0],[0, 0, 0]])]

    for R, t in poses:
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = t.T
        current_pose = np.dot(current_pose, T)
        trajectory.append(current_pose[:3, 3])
        pose_cam.append(current_pose[:3, :3])

    return np.array(trajectory),np.array(pose_cam)

In [66]:
trajectory, pose_cam = create_trajectory(poses)

In [67]:
import plotly.graph_objects as go
import numpy as np
# Создание фигуры
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=trajectory[1:, 0],
    y=trajectory[1:, 1],
    z=trajectory[1:, 2],
    mode='lines+markers',
    marker=dict(size=3, color='blue'),
    line=dict(color='blue', width=2),
    name='Camera Trajectory'
))

# Добавление ориентации камеры в каждой точке траектории
for i, (R, t) in enumerate(zip(pose_cam,trajectory)):
    # Направление камеры (ось Z камеры)
    camera_direction = R @ np.array([0, 0, 1])  # Направление оси Z камеры
    camera_direction_end = t + camera_direction * 0.5  # Конец вектора направления

    # Добавляем линию, представляющую направление камеры
    fig.add_trace(go.Scatter3d(
        x=[t[0], camera_direction_end[0]],
        y=[t[1], camera_direction_end[1]],
        z=[t[2], camera_direction_end[2]],
        mode='lines',
        line=dict(color='green', width=2),
        name=f'Camera Direction {i}' if i == 0 else None,
        showlegend=False if i > 0 else True
    ))

# Настройка макета графика
fig.update_layout(
    title='Camera Motion Trajectory',
    scene = dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    showlegend=True
)

# Показать график
fig.show()

В обоих случаях нейросеть некорректно отрисовала траекторию. В случае с увеличением или уменьшением интервала кадров может не отрисовывать ее вовсе. SIFT и ORB справились гораздо лучше

---